# Cleaning Data (Part 1)
The purpose of this notebook is to read in raw excel data for multiple years, append them into a single dataframe.

<a id='TOC'></a>

# Table of Contents
1. [Setting up Python](#SettingUp)
    
    a. [Setting the Location](#SettingLoc)
    
    b. [Importing Necessary Packages](#ImportingPackages)
    
    c. [Preparing for a Save](#PreparingSave)
    

2. [Reading in Data](#ReadingData)

3. [Cleaning Data](#CleaningData)

4. [Appending Data](#AppendingData)

5. [Exporting Data](#ExportingData)

<a id='SettingUp'></a>

# Setting up Python
[Top](#TOC)

[Setting the Location](#SettingLoc)
    
[Importing Necessary Packages](#ImportingPackages)
    
[Getting Data](#GettingData)
    
[Preparing for a Save](#PreparingSave)

<a id='ImportingPackages'></a>

## Importing Necessary Packages

[Top](#TOC)

[Setting Up Python](#SettingUp)

Here we import necessary packages. 
This chunk may take a while.

In [1]:
import pandas as pd
import numpy as np
import glob,os
from liz_number import lizsort,mindate,smallest,validate
from liz_toes import make_str,label_pattern, replace_pattern,report_pattern

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_config_file(world_readable=True)

# increase print limit
pd.options.display.max_rows = 99999

<a id='PreparingSave'></a>

## Preparing for a Save
[Top](#TOC)

[Setting up Python](#SettingUp)

<a id='SettingLoc'></a>

## Setting the Location
[Top](#TOC)

[Setting Up Python](#SettingUp)

These chunks identify the locations from which we can get data and to which we can save data.

### Source Data
Raw data can be found in the following locations:

In [2]:
# sourceDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
sourceDataBig = 'S:/Chris/TailDemography/TailDemography/Raw Data'
# sourceBlack = 'C:/Users/test/Desktop'

Now we change the working directory to the source path.

In [3]:
os.chdir(sourceDataBig)

### Output Data
The cleaned data will be saved to one of these locations:

In [4]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
outputBig = 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data'
# outputBlack = 'C:/Users/test/Desktop'

<a id='ReadingData'></a>

# Reading in Data
[Top](#TOC)

Here we use search the source path to locate and eventually read the raw data into our notebook.

In [5]:
rawfiles = glob.glob('*.xls*')
rawfiles

['CC 2000-03-modified from CC-SJ 00-03 final.xls',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2007x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx']

Let's inspect these data.  We will begin by looking at the number of columns and rows. In each file. To do this first we define a function, *xlsheets*, which we define below.

In [6]:
def xlsheets(file):
    """The function takes a string, *file*, and returns the number and names of sheets in that file.\
    The function relies on the pandas package."""
    tmp = pd.ExcelFile(file)
    res = {'num_sheets':len(tmp.sheet_names),'names': tmp.sheet_names}
    return res

You can see that when we call *xlsheets* on the first file in our list of files, it returns the total number of sheets and names of each sheet.

In [7]:
xlsheets(rawfiles[0])

{'num_sheets': 4, 'names': ['2000', '2001', '2002', '2003']}

We will apply this to all of the files in raw files to inspect the data at the highest level.

In [8]:
pd.Series(rawfiles).apply(xlsheets)

0     {'num_sheets': 4, 'names': ['2000', '2001', '2...
1                 {'num_sheets': 1, 'names': ['2004 ']}
2                  {'num_sheets': 1, 'names': ['2015']}
3                  {'num_sheets': 1, 'names': ['2016']}
4                  {'num_sheets': 1, 'names': ['2017']}
5                  {'num_sheets': 1, 'names': ['2005']}
6                  {'num_sheets': 1, 'names': ['2006']}
7        {'num_sheets': 2, 'names': ['Sheet1', '2007']}
8                  {'num_sheets': 1, 'names': ['2008']}
9                  {'num_sheets': 1, 'names': ['2009']}
10               {'num_sheets': 1, 'names': ['Sheet1']}
11               {'num_sheets': 1, 'names': ['Sheet1']}
12                 {'num_sheets': 1, 'names': ['data']}
13         {'num_sheets': 1, 'names': ['CC 2013 data']}
14                 {'num_sheets': 1, 'names': ['2014']}
dtype: object

We will need to treat the 1st and 8th files in the source data differently when we read in the data.  Let's group the file names accordingly.

In [9]:
rawfiles_ms = [rawfiles[0],rawfiles[7]]
rawfiles_ss = list(set(rawfiles)- set(rawfiles_ms))

The names of files with multiple sheets are now in the variable *rawfiles_ms*.

In [10]:
rawfiles_ms

['CC 2000-03-modified from CC-SJ 00-03 final.xls', 'xCC2007x.xls']

The names of files with a single sheet are now in the variable *rawfiles_ss*.

In [11]:
rawfiles_ss

['xCC2009x.xls',
 'xCC2012x.xls',
 'xCC2011x.xls',
 'xCC2014x.xlsx',
 'xCC2008x.xls',
 'xCC2006x.xls',
 'CC 2015 - captures.xls',
 'xCC2005x.xls',
 'CC 2004.xlsx',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2013x.xls',
 'xCC2010x.xlsx']

Now let's take a look at the number of columns in each file. We'll start with the single sheet files, since this is the easiest.  We will define another function, *xlcolshape* to make this easier.

In [12]:
def xlcolshape(file):
    """xlcolshape takes a file name as a string and returns the shape of the excel file"""
    return pd.read_excel(file).shape

When we call this function on the first of the single-sheet files, we can see that it returns a tuple in the format ('number of rows', 'number of columns').

In [13]:
xlcolshape(rawfiles_ss[0])

(162, 16)

We will apply this funtion to the list of single-sheet files for our inspection.

In [14]:
pd.Series(rawfiles_ss).apply(xlcolshape)

0     (162, 16)
1      (85, 19)
2      (64, 19)
3      (97, 19)
4     (134, 20)
5     (163, 16)
6     (241, 19)
7     (202, 16)
8     (479, 16)
9     (103, 21)
10    (798, 21)
11    (106, 19)
12     (99, 41)
dtype: object

<a id='CleaningData'></a>

# Cleaning Data
[Top](#TOC)

<a id='HandlingColumns'></a>

## Handling Columns
[Top](#TOC)

[Getting Data](#GettingData)

We don't have to look in the multiple-sheet file.  It's clear that we'll have to identify a common set of columns prior to combining these files.  Let's define a few functions to help us do this.

We will want to do the following:
1. [build a list of unique column names](#FindUniqueCol)
2. [eliminate unnecessary columns](#DropCol)
3. [combine synonyms ](#CombineCol)

<a id='FindUniqueCol'></a>

### Finding Unique Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

We'll define a function to extract column names and convert them to a set we'll use that function to allow us to only add unique names to a list of column names.

In [15]:
def xluniquecol(file,sheet=0):
    """xluniquecol takes a string filename of an excel file and extracts the column names as a set."""
    return list(pd.read_excel(file,sheet_name=sheet).columns)


Here is an example of how xluniquecol works.

In [16]:
xluniquecol(rawfiles_ss[0])

['Species',
 'Toes',
 'Date',
 'Sex',
 'SVL',
 'TL',
 'RTL',
 'Mass',
 'Paint Mark',
 'Location',
 'Meters',
 'New/Recap',
 ' painted or not',
 'Misc.',
 'Vial',
 'Time']

Now we will create an empty set, *uniquecols*, that will eventually contain the unique column names in all of the files.

We will append the unique column names from each file to *uniquecols*.

In [17]:
tmp = pd.Series(rawfiles_ss).apply(xluniquecol)
uniquecols = list()
for u in tmp:
    uniquecols = uniquecols+u
uniquecols = list(set(uniquecols))
uniquecols

['RTL',
 'Marked',
 1,
 'Toes',
 'paint mark',
 'Toe 20',
 'Year',
 'Paint Mark',
 'SVL',
 'Toe 12',
 'Painted',
 'species',
 'Mark',
 'Toe 15',
 'Toe 8',
 'Tail condition (1=intact; 2=autotomized; 3=regrown)',
 'NEW/recap',
 'SVL (mm)',
 '2015 or earlier',
 'mass',
 'TOES',
 'Toe 10',
 'Toe 16',
 'sex',
 'Mass',
 'Meters',
 'mass (g)',
 'misc/notes',
 'painted or not',
 'Collectors',
 '1st Capture (year)',
 'Toe 6',
 'Species',
 ' painted or not',
 'meters',
 'Toe 2',
 'painted',
 'Toe 17',
 'date',
 'Toe 18',
 'Location',
 'Toe 3',
 'Toe 9',
 'misc',
 'New/Recap',
 'Toe 14',
 'Years Alive (known)',
 'Misc.',
 'location',
 'Toe 7',
 'Toe 13',
 'RTL (mm)',
 'Date',
 'Unnamed: 17',
 'TL (mm)',
 'Unnamed: 16',
 'Toe 11',
 'Spotted',
 'TIME',
 'Toe 1',
 'Unnamed: 19',
 'Toe 19',
 'VIAL',
 'Toe 5',
 'TL',
 'Sex',
 'Time',
 'Vial',
 'Toe 4']

<a id='DropCol'></a>

### Eliminate Unnecessary Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Now we will try to identify unnecessary columns and eliminate them. Much of this will be done manually.

In [18]:
keepCol = ['species', 'date', 'sex', 'svl', 'tl', 'rtl', 'mass',
       'paint.mark', 'location', 'meters', 'new.recap', 'painted', 'misc',
       'vial', 'year', 'autotomized', 'sighting', 'toes']

In [19]:
set(pd.Series(keepCol).str.lower())-set(pd.Series(uniquecols).str.lower())

{'autotomized', 'new.recap', 'paint.mark', 'sighting'}

In [20]:
set(pd.Series(uniquecols).str.lower())-set(pd.Series(keepCol).str.lower())

{' painted or not',
 '1st capture (year)',
 '2015 or earlier',
 'collectors',
 'mark',
 'marked',
 'mass (g)',
 'misc.',
 'misc/notes',
 nan,
 'new/recap',
 'paint mark',
 'painted or not',
 'rtl (mm)',
 'spotted',
 'svl (mm)',
 'tail condition (1=intact; 2=autotomized; 3=regrown)',
 'time',
 'tl (mm)',
 'toe 1',
 'toe 10',
 'toe 11',
 'toe 12',
 'toe 13',
 'toe 14',
 'toe 15',
 'toe 16',
 'toe 17',
 'toe 18',
 'toe 19',
 'toe 2',
 'toe 20',
 'toe 3',
 'toe 4',
 'toe 5',
 'toe 6',
 'toe 7',
 'toe 8',
 'toe 9',
 'unnamed: 16',
 'unnamed: 17',
 'unnamed: 19',
 'years alive (known)'}

Since data for years 2000-2003 are contained in the same Excel file we will have to treat this file differently than the others.

<a id='CombineCol'></a>

### Combining Synonymous Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Once we have identified the columns we need to keep, we'll need to apply this list to the files as they are read into python by doing the following:

Let's define a function to identify potential synonyms. One approach is to take a column name that we want to keep and search through a list of names to identify any that contain enough of that name to be considered potentially synonymous.  Let's try to return these potential matches as values in a dictionary and return the desired name as the key for those values.

In [49]:
def colmatchtodict(x,series, dictsource, key= None):
    """This takes a string, x, and a looks for values in a series that match that contain that string.
    Those values which match are returned as values in a python dict for the key, key."""
    
    assert isinstance(series,pd.Series)
    if key is None:
        key = x
    tmp = series[series.astype(str).str.contains(x,case = False)].tolist()
    dictsource[key] = tmp
    return dictsource
    

In [57]:
coldict = {}

In [51]:
colmatchtodict('toes',pd.Series(uniquecols),coldict, key = 'toes')

{'toes': ['Toes', 'TOES']}

Now let's see what happened when we apply this funtion to our, keepCol.

In [60]:
coldict = {}

In [61]:
pd.Series(keepCol).apply(lambda x: colmatchtodict(x=x,series=pd.Series(uniquecols),dictsource=coldict))
coldict

{'species': ['species', 'Species'],
 'date': ['date', 'Date'],
 'sex': ['sex', 'Sex'],
 'svl': ['SVL', 'SVL (mm)'],
 'tl': ['RTL', 'RTL (mm)', 'TL (mm)', 'TL'],
 'rtl': ['RTL', 'RTL (mm)'],
 'mass': ['mass', 'Mass', 'mass (g)'],
 'paint.mark': ['paint mark', 'Paint Mark'],
 'location': ['Location', 'location'],
 'meters': ['Meters', 'meters'],
 'new.recap': ['NEW/recap', 'New/Recap'],
 'painted': ['Painted', 'painted or not', ' painted or not', 'painted'],
 'misc': ['misc/notes', 'misc', 'Misc.'],
 'vial': ['VIAL', 'Vial'],
 'year': ['Year', '1st Capture (year)', 'Years Alive (known)'],
 'autotomized': ['Tail condition (1=intact; 2=autotomized; 3=regrown)'],
 'sighting': [],
 'toes': ['Toes', 'TOES']}

Now we need to a

<a id='AppendingData'></a>

# Appending Data
[Top](#TOC)

<a id='ExportingData'></a>

# Exporting Data
[Top](#TOC)